In [1]:
#load path
import os
from astropy.io import fits
import glob
import shutil
from astroquery.astrometry_net import AstrometryNet


In [2]:
#astrometrically solve the images
def solve_astrometry_url(imagefile):
  ast = AstrometryNet()
  ast.api_key = 'ywslonowruohigzw' #'rqparxmlcjncjwnl' #

  wcshdu = fits.open(imagefile)
  try:
        haswcs = wcshdu[0].header['WCSAXES']
        print('Do not overwrite WCS for', imagefile)
  except KeyError:
        print('Process WCS for', imagefile)
        wcs_header = ast.solve_from_image(imagefile, force_image_upload=True, publicly_visible='n', solve_timeout=3600)
        if wcs_header:
            wcshdu = fits.open(imagefile)
            wcshdu[0].header += wcs_header
            wcshdu.writeto(imagefile, overwrite=True)
            print("Writing WCS to", imagefile)
        else:
            print('FAIL, WCS not modified')

In [3]:
#astrometrically solve the images
def solve_astrometry_local(imagefile, outdir='./'):
  import subprocess
  wcshdu = fits.open(imagefile)
  try:
        haswcs = wcshdu[0].header['WCSAXES']
        print('Do not overwrite WCS for', imagefile)
  except KeyError:
        print('Process WCS for', imagefile)
        shutil.copy(imagefile, outdir)
        subprocess.run('solve-field -L 0.2 -H 2.0 --no-plots --overwrite --dir {} {}'.format(outdir, outdir+os.path.basename(imagefile)), shell=True,stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        #copy the new wcs file back to the original name
        try:
            shutil.copy(outdir+os.path.basename(imagefile).replace('.fits','.new'), imagefile)
            print("Writing WCS to", imagefile)
        except:
            print('FAIL, WCS not modified')

In [4]:
def consolidate_objnames(imagefile):

    goodnames = ['NGC6946', 'NGC7331', 'M29', 'M39', 'M31', 'BD28', 'BD25', 'IC5146']

    print('Running', imagefile)
    hdu = fits.open(imagefile)
    objname = hdu[0].header['OBJECT']
    #check if objname is correct
    objfromfname = (imagefile.split('_')[3])
    if objname != objfromfname:
        hdu[0].header['OBJECT'] = objfromfname
        objname = hdu[0].header['OBJECT']
        hdu.writeto(imagefile, overwrite=True)
        print('Corrected OBJECT name in', imagefile, 'to', hdu[0].header['OBJECT'])

    #check if objname is written correctly
    if objname in goodnames:
        return
    else:
        editdict = {'ngc6946': 'NGC6946', 'NGC 6946': 'NGC6946', 'ngc_6946':'NGC6946', 'ngc7331': 'NGC7331', 'NGC_7331': 'NGC7331'}
        try:
            hdu[0].header['OBJECT'] = editdict[hdu[0].header['OBJECT']]
            hdu.writeto(imagefile, overwrite=True)
            print('Edited OBJECT name in', imagefile, 'to', hdu[0].header['OBJECT'])
            return
        except:
            print('No edit made for', imagefile, 'OBJECT name:', hdu[0].header['OBJECT'])
            return

In [7]:
path=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
#list directories in path
dirs = os.listdir(path)
dirs.sort()
print(dirs)

for dir in dirs:
    if os.path.isdir(path+dir+'/LIGHT/'):
        print('Now running astrometry for images in', dir+'/LIGHT/')
        newdir=path+dir+'/LIGHT/'
        lightimages = glob.glob(newdir+'/*sci*.fits')
        lightimages.sort()
        for fileout in lightimages:
            #consolidate_objnames(fileout)
            solve_astrometry_local(fileout, outdir='/Users/matteo/Teaching/labdataacq2025/data/Temp/')

['.DS_Store', '2025-06-30', '2025-07-02', '2025-09-07', '2025-09-17', '2025-09-25', '2025-09-28', '2025-09-29', '2025-09-30', '2025-10-01', '2025-10-02', '2025-10-06', '2025-10-07', '2025-10-09', '2025-10-10', '2025-10-15', '2025-10-16', '2025-10-18', '2025-10-22', '2025-10-24', '2025-10-26', '2025-10-30', '2025-11-03', '2025-11-04', '2025-11-05', '2025-11-07', '2025-11-08', '2025-11-10', '2025-11-11']
Now running astrometry for images in 2025-07-02/LIGHT/
Now running astrometry for images in 2025-09-07/LIGHT/
Now running astrometry for images in 2025-09-17/LIGHT/
Now running astrometry for images in 2025-09-28/LIGHT/
Do not overwrite WCS for /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-09-28/LIGHT/2025-09-28_19-26-28_sci_BD28_G_exp25.00_0000.fits
Do not overwrite WCS for /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-09-28/LIGHT/2025-09-28_19-26-54_sci_BD28_G_exp25.00_0001.fits
Do not overwrite WCS for /Users/matteo/Google Drive/My Drive/Telesco